In [1]:

from pyspark.sql import functions as sparkf

raw_df = spark.createDataFrame(
 [
  ('Store 1',1,448),
  ('Store 1',2,None),
  ('Store 1',3,499),
  ('Store 1',44,432),
  (None,None,None),
  ('Store 2',1,355),
  ('Store 2',1,355),
  ('Store 2',None,345),
  ('Store 2',3,387),
  ('Store 2',4,312)
],
 ['Store','WeekInMonth','Revenue']
)


In [2]:
raw_df.printSchema()

root
 |-- Store: string (nullable = true)
 |-- WeekInMonth: long (nullable = true)
 |-- Revenue: long (nullable = true)



In [3]:
raw_df.count()

raw_df.show()

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          1|    448|
|Store 1|          2|   null|
|Store 1|          3|    499|
|Store 1|         44|    432|
|   null|       null|   null|
|Store 2|          1|    355|
|Store 2|          1|    355|
|Store 2|       null|    345|
|Store 2|          3|    387|
|Store 2|          4|    312|
+-------+-----------+-------+



In [4]:
#พิจารณาจาก row count ในแต่ละ Attribute เทียบกับ row count จาก raw_df.count()

raw_df.describe().show()

+-------+-------+------------------+-----------------+
|summary|  Store|       WeekInMonth|          Revenue|
+-------+-------+------------------+-----------------+
|  count|      9|                 8|                8|
|   mean|   null|             7.375|          391.625|
| stddev|   null|14.841423689890979|62.74196021346939|
|    min|Store 1|                 1|              312|
|    max|Store 2|                44|              499|
+-------+-------+------------------+-----------------+



In [5]:
#แจงนับจำนวน Nulls ออกมาในแต่ละ Attribute

raw_df.select(
  [sparkf.count(sparkf.when(sparkf.isnull(c), c)).alias(c) for c in raw_df.columns]
).show()


+-----+-----------+-------+
|Store|WeekInMonth|Revenue|
+-----+-----------+-------+
|    1|          2|      2|
+-----+-----------+-------+



In [6]:
#ระบุว่า Nulls อยู่ที่ row ไหน

from functools import reduce

raw_df.filter(reduce(lambda a1, a2: a1 | a2,\
                     (sparkf.col(c).isNull() \
                      for c in raw_df.columns))).show()


+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          2|   null|
|   null|       null|   null|
|Store 2|       null|    345|
+-------+-----------+-------+



In [7]:
#เลือกเฉพาะ row ที่มีค่า nulls ในทุก attributes แล้วซ่อนค่า nulls จากนั้นส่งให้กับตัวแปรใหม่

noNullRow_Df = raw_df.dropna('all')

noNullRow_Df.show()

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          1|    448|
|Store 1|          2|   null|
|Store 1|          3|    499|
|Store 1|         44|    432|
|Store 2|          1|    355|
|Store 2|          1|    355|
|Store 2|       null|    345|
|Store 2|          3|    387|
|Store 2|          4|    312|
+-------+-----------+-------+



In [8]:
#เลือก row ที่มีค่า nulls ใน attributes ใด attribute หนึ่ง (หรือทุก attributes) จาก subset
#แล้วซ่อนค่า nulls

raw_df.dropna(how='any', \
              subset=['Store','WeekInMonth']).show()


+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          1|    448|
|Store 1|          2|   null|
|Store 1|          3|    499|
|Store 1|         44|    432|
|Store 2|          1|    355|
|Store 2|          1|    355|
|Store 2|          3|    387|
|Store 2|          4|    312|
+-------+-----------+-------+



In [9]:
#เลือก row ที่มีค่า nulls ใน attributes ใด attribute หนึ่ง (หรือทุก attributes) แล้วซ่อนค่า nulls

raw_df.dropna(how='any').show()

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          1|    448|
|Store 1|          3|    499|
|Store 1|         44|    432|
|Store 2|          1|    355|
|Store 2|          1|    355|
|Store 2|          3|    387|
|Store 2|          4|    312|
+-------+-----------+-------+



In [10]:
#แทนที่ค่า nulls ด้วยเลข 0 เฉพาะใน “Revenue”

raw_df.fillna(0,['Revenue']).show()

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          1|    448|
|Store 1|          2|      0|
|Store 1|          3|    499|
|Store 1|         44|    432|
|   null|       null|      0|
|Store 2|          1|    355|
|Store 2|          1|    355|
|Store 2|       null|    345|
|Store 2|          3|    387|
|Store 2|          4|    312|
+-------+-----------+-------+



In [11]:
#แทนที่ค่า nulls ด้วยเลข 0 ในทุก Attributes

raw_df.fillna(0).show()

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          1|    448|
|Store 1|          2|      0|
|Store 1|          3|    499|
|Store 1|         44|    432|
|   null|          0|      0|
|Store 2|          1|    355|
|Store 2|          1|    355|
|Store 2|          0|    345|
|Store 2|          3|    387|
|Store 2|          4|    312|
+-------+-----------+-------+



In [13]:
#แทนที่ค่า nulls ด้วยเลข 0 ในทุก Attributes ซึ่งเป็น numeric และด้วยค่าอื่นๆ สำหรับ string
raw_df.fillna({'Store':'Assume_Store 1',\
              'WeekInMonth':'2','Revenue':3}).show()


+--------------+-----------+-------+
|         Store|WeekInMonth|Revenue|
+--------------+-----------+-------+
|       Store 1|          1|    448|
|       Store 1|          2|      3|
|       Store 1|          3|    499|
|       Store 1|         44|    432|
|Assume_Store 1|          2|      3|
|       Store 2|          1|    355|
|       Store 2|          1|    355|
|       Store 2|          2|    345|
|       Store 2|          3|    387|
|       Store 2|          4|    312|
+--------------+-----------+-------+



In [14]:
from pyspark.ml.feature import Imputer

In [15]:
correctedType_df = raw_df\
.withColumn('RevenueD',sparkf.col('Revenue').cast('double'))
imputer = Imputer(
    inputCols=['RevenueD'],
    outputCols=['RevenueD'],
    strategy='median'
)


In [16]:
imputer.fit(correctedType_df\
            .filter(sparkf.col('Store') == 'Store 1'))\
.transform(correctedType_df).show()


+-------+-----------+-------+--------+
|  Store|WeekInMonth|Revenue|RevenueD|
+-------+-----------+-------+--------+
|Store 1|          1|    448|   448.0|
|Store 1|          2|   null|   448.0|
|Store 1|          3|    499|   499.0|
|Store 1|         44|    432|   432.0|
|   null|       null|   null|   448.0|
|Store 2|          1|    355|   355.0|
|Store 2|          1|    355|   355.0|
|Store 2|       null|    345|   345.0|
|Store 2|          3|    387|   387.0|
|Store 2|          4|    312|   312.0|
+-------+-----------+-------+--------+

